In [1]:
from math import ceil

import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import DenseGraphConv, GCNConv, dense_mincut_pool
from torch_geometric.utils import to_dense_adj, to_dense_batch

import sys
import numpy as np

sys.path.append("../")
import utils
import torch_geometric as tg

from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score, accuracy_score

In [2]:
early_stop_thresh = 25
best_macro_f1 = -1


train_dataset = utils.GraphDataset("../data/", "GunPoint", True)
train_loader = DataLoader(train_dataset, batch_size=50, shuffle=True)

test_dataset = utils.GraphDataset("../data/", "GunPoint", False)
test_loader = DataLoader(test_dataset, batch_size=50)

In [3]:
class Net(torch.nn.Module):
    def __init__(
        self,
        in_channels,
        out_channels,
        graph_sizes,
        n_edge_layers=2,
        hidden_channels=32,
    ):
        super().__init__()
        self.edge_layers = torch.nn.ModuleList()
        for it in range(n_edge_layers):
            nn = tg.nn.MLP([in_channels, hidden_channels, hidden_channels])
            self.edge_layers.append(tg.nn.GINConv(nn))
            in_channels = hidden_channels
        self.gcn_layers = torch.nn.ModuleList()
        for n_nodes in graph_sizes:
            self.gcn_layers.append(Linear(hidden_channels, n_nodes))
            nn = tg.nn.MLP([hidden_channels, hidden_channels])
            self.gcn_layers.append(tg.nn.DenseGINConv(nn))
            nn = tg.nn.MLP([hidden_channels, hidden_channels])
            self.gcn_layers.append(tg.nn.DenseGINConv(nn))
        nn = tg.nn.MLP([hidden_channels, hidden_channels])
        self.gcn_layers.append(tg.nn.DenseGINConv(nn))

        self.lin1 = Linear(hidden_channels, hidden_channels)
        self.lin2 = Linear(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index, edge_attr, batch = (
            data.x,
            data.edge_index,
            data.edge_attr,
            data.batch,
        )
        for layer in self.edge_layers:
            x = layer(x, edge_index).relu()
        x, mask = to_dense_batch(x, batch)
        adj = to_dense_adj(edge_index, batch)

        mincut_loss = torch.zeros(1)
        ortho_loss = torch.zeros(1)
        for layer in self.gcn_layers:
            if layer._get_name() == "Linear":
                s = layer(x)
                x, adj, mc, o = dense_mincut_pool(x, adj, s, mask)
                mincut_loss += mc
                ortho_loss += o
                mask = None
            else:
                x = layer(x, adj).relu()
        x = x.mean(dim=1)
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1), mincut_loss, ortho_loss

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net(1, 5, [50, 20], hidden_channels=16).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min", cooldown=2, factor=0.5, verbose=True
)

In [5]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        y_out, mc_loss, o_loss = model(data)
        loss = F.nll_loss(y_out, data.y) + mc_loss + o_loss
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out, mc_loss, o_loss = model(data)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y)
        loss += float(F.nll_loss(y_out, data.y) * data.num_graphs + mc_loss + o_loss)
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    return (
        f1_score(y_true=y_true, y_pred=y_pred, average="macro"),
        accuracy_score(y_true=y_true, y_pred=y_pred),
        loss / len(loader.dataset),
    )


best_val_acc = test_acc = 0
best_val_loss = float("inf")
patience = start_patience = 50

In [6]:
for epoch in range(1, 1001):
    train()
    train_macro_f1, train_acc, train_loss = test(train_loader)
    test_macro_f1, test_acc, test_loss = test(test_loader)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f},Test_Loss: {test_loss:02.4f},Train_f1: {train_macro_f1:01.4f},Test_f1: {test_macro_f1:01.4f},Train_acc: {train_acc:01.4f},Test_acc: {test_acc:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")

Epoch: 001, Train_Loss: 1.6860,Test_Loss: 1.6807,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 002, Train_Loss: 1.6721,Test_Loss: 1.6665,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 003, Train_Loss: 1.6592,Test_Loss: 1.6532,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 004, Train_Loss: 1.6480,Test_Loss: 1.6417,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 005, Train_Loss: 1.6389,Test_Loss: 1.6323,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 006, Train_Loss: 1.6302,Test_Loss: 1.6233,Train_f1: 0.0000,Test_f1: 0.0000,Train_acc: 0.0000,Test_acc: 0.0000
Epoch: 007, Train_Loss: 1.6237,Test_Loss: 1.6165,Train_f1: 0.2222,Test_f1: 0.2282,Train_acc: 0.4800,Test_acc: 0.5067
Epoch: 008, Train_Loss: 1.6180,Test_Loss: 1.6107,Train_f1: 0.3243,Test_f1: 0.3363,Train_acc: 0.4800,Test_acc: 0.5067
Epoch: 009, Train_Loss: 1.6128,Test_Loss: 1.6053,Train_f1: 0.324